# Asana Yearly Task Archival Notebook

**Problem:** You have an Asana project board that accumulates tasks over the year. At the end of the year, you want to move all completed tasks from this year to another project called "Archive Board." This script helps with the archive process.

To begin, import the following libraries. If you are missing a library, complete the following steps: 
1. Open a local terminal session, and run `pip3 install <library_name>`. 
1. Once the library is installed, click **Kernel** > **Restart**.

In [ ]:
import asana
import csv
import pandas as pd

## Personal Access Token
Obtain your personal access token via Asana. Replace `<INSERT_ASANA_TOKEN>` with this token, and keep it wrapped in quotes. See the "Personal Access Tokens" section of the [Asana Developer Docs](https://asana.com/guide/help/api/api) for information on how to create and obtain one. 

In [ ]:
personal_access_token ="<INSERT_ASANA_TOKEN>"

## Getting Section IDs

Use the [API Reference](https://developers.asana.com/reference/rest-api-reference) to find the IDs for boards and sections. 

> **IMPORTANT**: If you use the same archive board each year with tasks for that year separated by section, you will need to add a new section to the Archive board for that particular year. Then, obtain that section's ID.

1. From the left navigation menu, click **Sections** > **Get sections in a project**. 
1. For `project_gid`, enter the project board's ID. To obtain the project ID, go to the Asana board for that project. The ID is found in the URL.
1. For **Authorization**, select **Bearer** and add your personal access token.
1. Click **Try It!**. Observe the JSON response with all section names and IDs for that board. Replace the placeholders in this notebook with the appropriate IDs.

# Functions
Run these cells to store the functions in memory.

## Obtain All Tasks from Original Board Location

In [ ]:
def get_all_tasks(output_file, original_project):
    '''Obtains all tasks for project and write to a csv file defined in the function variables
        You can use the csv file to track all tasks that were moved in case of errors'''
    file = open(output_file, "w")
    fieldnames= ["gid", "name", "completed"]
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()

    result = client.tasks.get_tasks_for_project(original_project, opt_fields=["name", "completed"], opt_pretty=True)

    writer.writerows(list(result))
    file.close()

## Clean CSV File to Show Only Completed Tasks

In [ ]:
def clean_task_file(input_file, output_file):
    '''Opens created csv and creates a new, cleaned csv with only completed tasks'''
    df = pd.read_csv(input_file)
    df_clean = df[df["completed"] == True]
    df_clean.to_csv(output_file)

## Move to Corresponding Archive Project

In [ ]:
def archive_tasks(input_file, archive_project, archive_section, original_project):
    '''Adds Archive project to each task as well as the correct section and removes it from the original project '''
    file = input_file
    rows = []
    
    with open(file, "r") as csvfile:
        csvreader = csv.reader(csvfile)
        fields = next(csvreader) #remove headers
        for row in csvreader:
            rows.append(row)

    for row in rows:
        update_task_project = client.tasks.add_project_for_task(row[1], {"project": archive_project, "section": archive_section}, opt_pretty=True)
        remove_project = client.tasks.remove_project_for_task(row[1], {"project": original_project}, opt_pretty=True)


# Archive Process

In [ ]:
original_project = "<ORIGINAL_PROJECT_ID" 
archive_project = "<ARCHIVE_PROJECT_ID>"
archive_section = "<ARCHIVE_SECTION_ID" #You will need to add a new section id for the year 

## Obtain All Tasks from Original Project Board and Clean File

In [ ]:
client = asana.Client.access_token(personal_access_token)
get_all_tasks("tasks.csv", original_project)
clean_task_file("tasks.csv", "done_tasks_cleaned.csv")

## Add the Archive Project Board and Remove from Original Board

In [ ]:
archive_tasks("done_tasks_cleaned.csv", archive_project, archive_section, original_project)